First we will be setting up all imports and set up basic logging:

In [ ]:
from memflow import *
from ctypes import *
import logging

FORMAT = "%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s"
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)

The first step towards memflow is then creating an Inventory. The Inventory contains a list of all connectors found on the System:

In [ ]:
inventory = Inventory()

The next step is creating an actual connector. Memflow connectors are used to acquire physical memory of a device. Examples of connectors are: kvm, qemu, pcileech, coredump, ...

The arguments are provided as triplet, they use the following form:

```
{connector}:{target}:{arg1},{arg2},{arg3}
```

Where `target` is the name of the target (in case the connector supports multiple targets).
Where `arg1`, `arg2` and `arg3` use a `key=value` format like `device=FPGA`

Here we are loading the `kvm` connector and letting it connect to a virtual machine with the name `win11` as the target.

In [ ]:
conn = inventory.connector("kvm", ":win11:")


The next step is to utilize the previously created connector to initialize an OS. In the given example we try to find Windows running in memory. 

In [ ]:
os = inventory.os("win32", conn)

You can now access drivers and processes on the target. In the windows (and memflow) world drivers are just modules of the root process (in the case of Windows all drivers are modules of the ntoskrnl.exe kernel process). The following example shows how to list all running drivers:

In [ ]:
from pprint import pprint

drivers = os.module_info_list()
for driver in drivers: # TODO: implement str for module list
    pprint(driver.name)

To access the memory of a driver or process you have to open the corresponding process:

In [ ]:
kernel = os # TODO:
process = os.process_from_name("explorer.exe")

In [ ]:
from pprint import pprint

# List all modules in the process:
modules = process.module_info_list()
for mod in modules: # TODO: implement str for module list
    pprint(mod.name)

In [ ]:
# Load module 'Explorer.EXE':
module = process.module_from_name("Explorer.EXE")

In [ ]:
# Define COFF Header struct and read it from the module:
class COFFHeader(Structure):
    _fields_ = [
        ("_pad0x0", c_uint8 * 6),
        ("sections", c_uint16),
        ("timestamp", c_uint32),
    ]

    def __str__(self):
        return f"{self.sections}, {self.timestamp}"

header = process.read(module.base + 0x40, COFFHeader)
print(header)